In [23]:
import random
import torch

import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable
import numpy as np
from warpctc_pytorch import CTCLoss
import os
import crnn.utils as utils
import crnn.dataset as dataset
import crnn.crnn as crnn

In [24]:
dset = dataset.lmdbDataset(root='../data/train/telephone/tele_data/')
dataloader = DataLoader(dset, batch_size=32, num_workers=4, collate_fn=dataset.alignCollate())

In [25]:
nh = 100
alphabet = '0123456789'
nclass = len(alphabet) + 1
nc = 1

In [26]:
converter = utils.strLabelConverter(alphabet)
criterion = CTCLoss()

In [27]:
crnn = crnn.CRNN(32, nc, nclass, nh)

In [28]:
crnn.cuda()
image = image.cuda()
criterion = criterion.cuda()

In [29]:
loss_avg = utils.averager()
optimizer = optim.Adadelta(crnn.parameters(), lr=1e-3)

In [ ]:
def trainBatch(net, criterion, optimizer):
    data = train_iter.next()
    cpu_images, cpu_texts = data
    batch_size = cpu_images.size(0)
    image = torch.FloatTensor(cpu_images)
    image = Variable(image).cuda()
    t, le = converter.encode(cpu_texts)
    text = torch.IntTensor(t)
    text = Variable(text)
    length = torch.IntTensor(le)
    length = Variable(length)
    preds = crnn(image)
    preds_size = Variable(torch.IntTensor([preds.size(0)] * batch_size))
    cost = criterion(preds, text, preds_size, length) / batch_size
    crnn.zero_grad()
    cost.backward()
    optimizer.step()
    return cost


for epoch in range(10000):
    train_iter = iter(dataloader)
    i = 0
    while i < len(dataloader):
        for p in crnn.parameters():
            p.requires_grad = True
        crnn.train()

        cost = trainBatch(crnn, criterion, optimizer)
        loss_avg.add(cost)
        i += 1

        if i % 10 == 0:
            print('[%d/%d][%d/%d] Loss: %f' %
                  (epoch, 10, i, len(dataloader), loss_avg.val()))
            loss_avg.reset()

#         if i % opt.valInterval == 0:
#             val(crnn, test_dataset, criterion)

        # do checkpointing
#         if i % opt.saveInterval == 0:
#             torch.save(
#                 crnn.state_dict(),
#                 '{0}/netCRNN_{1}_{2}.pth'.format(opt.experiment, epoch, i))


In [66]:
import PIL.Image as Image
img_path = '../data/train/telephone/telephone/18.png'
transformer = dataset.resizeNormalize((128, 32))
image = Image.open(img_path).convert('L')
image = transformer(image).cuda()
image = image.view(1, *image.size())
image = Variable(image)

crnn.eval()
preds = crnn(image)
_, preds = preds.max(2)
preds = preds.squeeze(2)
preds = preds.transpose(1, 0).contiguous().view(-1)
preds_size = Variable(torch.IntTensor([preds.size(0)]))
raw_pred = converter.decode(preds.data, preds_size.data, raw=True)
sim_pred = converter.decode(preds.data, preds_size.data, raw=False)
print('%-20s => %-20s' % (raw_pred, sim_pred))


118882--22---55-----11111244111-1 => 1822512411          
